# Wielomiany nad ciałem skończonym

Ten zestaw jest przygotowany w oparciu o rozdział 14 książki "Modern computer algebra" Joachima von zur Gathena i Jurgena Gerharda.

Niech $\mathbb{F}_q$ będzie ciałem $q$-elementowym, gdzie $q$ jest potęgą liczby pierwszej. Konstrukcja tego ciała pojawiła się m.in. w części trzeciej zestawu "Permutacje, generatory i grupy cykliczne". Przypomnijmy, że dla ustalonego $q$ wszystkie ciała $q$-elementowe są takiej samej postaci (bardziej precyzyjnie: są ze sobą izomorficzne). W SageMath ciało $\mathbb{F}_q$ jest już zdefiniowane za pomocą komendy `GF(q)`. 

Na przykład:

In [1]:
show(list(GF(3))) # lista elementów ciała 3-elementowego
show(list(GF(9))) # lista elementów ciała 9-elementowego
show(list(GF(27))) # lista elementów ciała 27-elementowego

[0, 1, 2]

[0, z2, z2 + 1, 2*z2 + 1, 2, 2*z2, 2*z2 + 2, z2 + 2, 1]

[0,
 z3,
 z3^2,
 z3 + 2,
 z3^2 + 2*z3,
 2*z3^2 + z3 + 2,
 z3^2 + z3 + 1,
 z3^2 + 2*z3 + 2,
 2*z3^2 + 2,
 z3 + 1,
 z3^2 + z3,
 z3^2 + z3 + 2,
 z3^2 + 2,
 2,
 2*z3,
 2*z3^2,
 2*z3 + 1,
 2*z3^2 + z3,
 z3^2 + 2*z3 + 1,
 2*z3^2 + 2*z3 + 2,
 2*z3^2 + z3 + 1,
 z3^2 + 1,
 2*z3 + 2,
 2*z3^2 + 2*z3,
 2*z3^2 + 2*z3 + 1,
 2*z3^2 + 1,
 1]

Widzimy, że ciało $\mathbb{F}_{p^n}$ składa się z wszystkich możliwych wielomianów stopnia mniejszego od $n$ o współczynnikach z ciała $\mathbb{F}_p=\mathbb{Z}_p$. Możemy narzucić nazwę zmiennej występującej w tych wielomianach w następujący sposób:

In [2]:
R.<a>=GF(9)
list(R)

[0, a, a + 1, 2*a + 1, 2, 2*a, 2*a + 2, a + 2, 1]

Przypomnijmy, że aby zdefiniować pierścień wielomianów o współczynnikach z ciała $\mathbb{F}_q$, a następnie elemety tego pierścienia, możemy użyć następujących komend:

In [10]:
q=5
R.<x>=PolynomialRing(GF(q))
f=x^4 + 2*x^3 + 2*x^2 + 3
f

x^4 + 2*x^3 + 2*x^2 + 3

## 1. Algorytmy wstępne

**Zadanie 1.1**  Dostosuj algorytm szybkiego potęgowania z poprzedniego zestawu tak, by dla wielomianów $a,f\in\mathbb{F}_q[x]$ oraz liczby naturalnej $e$ obliczyć $a^e$ modulo $f$. Każde mnożenie wykonuj modulo $f$.

**Sprawdzenie:** $\quad$ dla $\quad q=5$, $\quad a=x^3 + 2x^2 + 1$, $\quad e=5$, $\quad f=x^4 + 2x^3 + 2x^2 + 3\quad$ mamy $\quad a^e=x^2 + 2x + 3\pmod f$.

## 2. Faktoryzacja wielomianu nad ciałem skończonym

Ustalmy wielomian $f\in\mathbb{F}_q[x]$ stopnia $n\geq 1$. Załóżmy najpierw, że $f$ jest **wielomianem bezkwadratowym**, tzn. że $f$ nie jest podzielny przez kwadrat żadnego wielomianu. Faktoryzacja bezkwadratowego wielomianu $f$ składa się z dwóch kroków:  
1. distinct degree factorization: dla każdego $d\leq n$ znajdujemy wielomian $g$, który jest iloczynem wszystkich różnych nierozkładalnych wielomianów stopnia $d$ dzielących wielomian $f$;
2. equal degree factorization: dla wielomianu $g$ postaci $g=f_1\cdot\ldots\cdot f_s$, gdzie $f_1,\ldots ,f_s$ są różnymi nierozkładalnymi wielomianami tego samego stopnia $d$, znajdujemy wielomiany $f_1,\ldots ,f_s$.

Algorytmy distinct degree factorization i equal degree factorization połączone w odpowiedni sposób pozwalają przeprowadzić faktoryzację dowolnego wielomianu - zobacz krok 3 poniżej.

### Krok 1: distinct degree factorization  

Ten krok bazuje na następującym twierdzeniu.  

**Twierdzenie.** Niech $d\in\mathbb{N}$. Wielomian $x^{q^d}-x\in\mathbb{F}_q[x]$ jest iloczynem wszystkich nierozkładalnych unormowanych wielomianów z $\mathbb{F}_q[x]$, których stopień dzieli $d$.

Przypomnijmy, że wielomian $f(x)=a_n x^n+\ldots a_1 x+a_0$ jest **unormowany**, gdy $a_n=1$.

Algorytm wykonujący distinct degree factorization można opisać za pomocą następującego pseudokodu:  

wejście: wielomian bezkwadratowy unormowany $f\in\mathbb{F}_q[x]$ stopnia $n>0$  

$h_0 = x$, $\quad f_0=f$, $\quad i=0$  

jak długo $f_i\neq 1$:  

$\qquad\quad i=i+1$  
        
$\qquad\quad h_i=h_{i-1}^q$ modulo $f$  
        
$\qquad\quad g_i=\mathrm{NWD}(h_i - x, f_{i-1})$, $\quad f_i=\frac{f_{i-1}}{g_i}$  
        
wyjście: $(g_1, g_2, \ldots )$  

Zauważmy, że każdy wielomian $g_i$ otrzymany powyżej jest iloczynem wszystkich parami różnych wielomianów nierozkładalnych stopnia $i$, które dzielą wielomian $f$. W szczególności, $g_i$ jest wielomianem bezkwadratowym.

**Zadanie 2.1** Wykonaj distinct degree factorization dla następujących wielomianów:

**a).** $f(x)=x^{17}+2x^{15}+4x^{13}+x^{12}+2x^{11}+2x^{10}+3x^9+4x^8+4x^4+3x^3+2x^2+4x\in\mathbb{F}_{5}[x]$

In [ ]:
q=5
R.<x>=PolynomialRing(GF(q))
f=x^(17)+2*x^(15)+4*x^(13)+x^(12)+2*x^(11)+2*x^(10)+3*x^9+4*x^8+4*x^4+3*x^3+2*x^2+4*x


**b).** $f(x)=x^8 + x^7 - x^6 + x^5 - x^3 - x^2 - x\in\mathbb{F}_3[x]$

In [ ]:
q=3
R.<x>=PolynomialRing(GF(q))
f=x^8+x^7-x^6+x^5-x^3-x^2-x


**Sprawdzenie:** $\quad$ Jak łatwo można sprawdzić czy uzyskane wyniki są poprawne?

**Uwaga.** Napisanie funkcji, która wykonuje distinct degree factorization dla wielomianu $f$ wymaga znajomości zmiennej tego wielomianu oraz ciała, nad którym mają być wykonywane obliczenia. W konsekwencji, kod napisany powyżej nie będzie działał dla wielomianu zmiennej $t$ lub jakiejkolwiek innej zmiennej różnej od $x$. Za każdym razem trzeba bedzie też osobno podać $q$, liczbę elementów ciała $\mathbb{F}_q$, nad którym wykonywane są obliczenia. Aby sobie z tym poradzić i zaimplementować ogólny algorytm, można postąpić na kilka sposobów. 

Informacje o zmiennej wielomianu $f$ oraz o ciele, nad którym jest zdefiniowany, można uzyskać z $f$ za pomocą następujących komend:

In [ ]:
p=(f.base_ring()).order()
a=f.variables()[0]
K=PolynomialRing(GF(p),a)

Na przykład, aby zdefiniowanć wielomian $g$ tej samej zmiennej co $f$ i obliczyć $\mathrm{NWD}(g,f)$ można postąpić w następujący sposób:

In [3]:
q=9
R.<x>=PolynomialRing(GF(q))
f=x^8+x^7-x^6+x^5-x^3-x^2-x
p=(f.base_ring()).order()
a=f.variables()[0]
K=PolynomialRing(GF(p),a)
g=a^2-a
gcd(g,f)

x

Można też uczynić kod bardziej interaktywnym i poprosić użytkownika aby sam podał zmienną wielomianu, $q$ oraz wielomian, dla którego chciałby przeprowadzić faktoryzację:

In [32]:
f = input("Podaj wielomian, dla którego ma zostać wykonana faktoryzacja: ")
x = input("Podaj zmienną wielomianu: ")
q = ZZ(input("Podaj liczbę elementów ciała, nad którym wielomian jest zdefiniowany: "))
R=PolynomialRing(GF(q),x)
x=R(x) # dzięki temu x jest traktowany jako element pierścienia R
f=R(f)
f*(x+1) # sprawdzenie czy można pomnożyć f przez wielomian zmiennej x

Podaj wielomian, dla którego ma zostać wykonana faktoryzacja:t^2+2*t+1
Podaj zmienną wielomianu: t
Podaj liczbę elementów ciała, nad którym wielomian jest zdefiniowany: 9


t^3 + 1

### Zastosowanie: badanie nierozkładalności wielomianu nad ciałem skończonym

Przypomnijmy, że wielomian $f$ jest nierozkładalny jeśli nie istnieją wielomiany $g$ i $h$ stopnia co najmniej $1$ takie, że $f=gh$. 

**Zadanie 2.2** Korzystając z funkcji z zadania 2.1, rozstrzygnij czy poniższe wielomiany są rozkładalne czy nie.

**a).** $f(x)=5x^6 + 6x^5 + 4x^3 + 2x + 2\in\mathbb{F}_7[x]$

In [ ]:
q=7
R.<x>=PolynomialRing(GF(q))
f=5*x^6 + 6*x^5 + 4*x^3 + 2*x + 2


**b).** $f(x)=3x^5 + 3x^4 + 5x^3 + 4x^2 + 4x + 1\in\mathbb{F}_{13}[x]$

In [ ]:
q=13
R.<x>=PolynomialRing(GF(q))
f=3*x^5 + 3*x^4 + 5*x^3 + 4*x^2 + 4*x + 1


**Sprawdzenie:** $\quad$ Jeśli masz wątpliwości co do swojego wyniku, wykorzystaj funkcję SageMath `factor()`.

### Krok 2: equal degree factorization (algorytm Cantora i Zassenhausa)

Krok 2. faktoryzacji wielomianu różni się w zależności od tego czy dla ciała $\mathbb{F}_q$, gdzie $q=p^n$, mamy $p=2$ czy $p\neq 2$.

Załóżmy najpierw, że $p\neq 2$. Niech $f\in\mathbb{F}_q[x]$ będzie wielomianem unormowanym bezkwadratowym stopnia $n>0$, który jest iloczynem wielomianów nierozkładalnych tego samego stopnia $d$.  

Jeśli $n=d$, to $f$ jest wielomianem nierozkładalnym.  

Jeśli $d<n$, to powtarzając następującą probabilistyczną procedurę wystarczająco długo, znajdziemy wielomian $g\in\mathbb{F}_q[x]$, który dzieli $f$:  
1. losujemy $a\in\mathbb{F}_q[x]$; $\quad$ jeśli $a\in\mathbb{F}_q$, zwróć $0$  
2. niech $g_1=\mathrm{NWD}(a,f)$; $\quad$ jeśli $g_1\neq 1$, zwróć $g_1$  

3. oblicz $b= a^{(q^d-1)/2}$ modulo $f$  

4. niech $g_2=\mathrm{NWD}(b-1,f)$;  
jeśli $g_2\neq 1$ i $g_2\neq f$, zwróć $g_2$, w przeciwnym wypadku zwróć $0$.  

Znaleźliśmy wielomian $g$, który dzieli $f$. Możemy zastosować powyższą procedurę do wielomianów $g$ i $f/g$. Powtarzamy ją rekurencyjnie dla każdej pary wielomianów aż znajdziemy wszystkie dzieliniki stopnia $d$ wielomianu $f$. 

**Zadanie 2.3** (algorytm Cantora i Zassenhausa)  

**a).** Zaimplementuj punkty 1 - 4 opisane powyżej.

**b).** Niech $f\in\mathbb{F}_q$, gdzie $2\nmid q$, będzie wielomianem unormowanym bezkwadratowym stopnia $n>0$, który jest iloczynem wielomianów nierozkładalnych tego samego stopnia $d$. Zaimplementuj procedurę opisaną powyżej do znalezienia wszystkich wielomianów stopnia $d$ dzielących $f$.

**Zadanie 2.4** Wykorzystaj funkcje (lub ich fragmenty) z zadań 2.1 i 2.2 do znalezienia pierwiastków (o ile istnieją) wielomianów z zadania 2.1.

In [2]:
q=5
R.<x>=PolynomialRing(GF(q))
f=x^(17)+2*x^(15)+4*x^(13)+x^(12)+2*x^(11)+2*x^(10)+3*x^9+4*x^8+4*x^4+3*x^3+2*x^2+4*x


In [ ]:
q=3
R.<x>=PolynomialRing(GF(q))
f=x^8+x^7-x^6+x^5-x^3-x^2-x


**Sprawdzenie:** $\quad$ Wykorzystaj funkcje SageMath `roots()` lub `factor()`.

### Krok 3: faktoryzacja dowolnego wielomianu $f$

Niech $f(x)=a_n x^n+\ldots +a_1 x+a_0\in\mathbb{F}_q[x]$, $q$ nieparzyste. Powyżej opisane procedury można połączyć jak następuje, aby uzyskać pełną faktoryzację wielomianu $f$. 

$h_0 = x$, $\quad f_0=\frac{f}{a_n}$, $\quad i=0$,  $\quad U=\emptyset$   

jak długo $f_0\neq 1$:  

$\qquad\quad i=i+1$  
        
$\qquad\quad h_i=h_{i-1}^q$ modulo $f$  
        
$\qquad\quad g_i=\mathrm{NWD}(h_i - x, f_{0})$ 

$\qquad\quad$ jeśli $g_i\neq 1$, znajdź wszystkie wielomiany nierozkładalne $k_1,\ldots , k_s\in\mathbb{F}_q[x]$ stopnia $i$, które dzielą $g_i$ (equal degree factorization)

$\qquad\quad$ dla każdego $j\in\{ 1\ldots ,s\}$:   
$\qquad\quad\qquad\quad e=0$  
$\qquad\quad\qquad\quad f_0=\frac{f_{0}}{k_j}$, $\quad e=e+1$  
$\qquad\quad\qquad\quad U=U\cup \{ (k_j,e)\}$

zwróć $U$

**Zadanie 2.5** Znajdź faktoryzację następujących wielomianów.

**a).** $f(x)=x^{17}+2x^{15}+4x^{13}+x^{12}+2x^{11}+2x^{10}+3x^9+4x^8+4x^4+3x^3+2x^2+4x\in\mathbb{F}_{5}[x]$

In [ ]:
q=5
R.<x>=PolynomialRing(GF(q))
f=x^(17)+2*x^(15)+4*x^(13)+x^(12)+2*x^(11)+2*x^(10)+3*x^9+4*x^8+4*x^4+3*x^3+2*x^2+4*x


**b).** $f(x)=x^8 + x^7 - x^6 + x^5 - x^3 - x^2 - x\in\mathbb{F}_3[x]$

In [ ]:
q=3
R.<x>=PolynomialRing(GF(q))
f=x^8+x^7-x^6+x^5-x^3-x^2-x


**c).** $f(x)=x^{20} + 6x^{19} + 2x^{18} + x^{17} + 3x^{16} + 5x^{15} + 2x^{14} + 6x^{13} + 3x^{12} + 5x^{11} + 2x^{10} + 4x^9 + 4x^7 + 5x^6 + 3x^4 + 3x^3 + 5x^2 + 5x + 6\in\mathbb{F}_7[x]$

In [ ]:
q=7
R.<x>=PolynomialRing(GF(q))
f=x^(20) + 6*x^(19) + 2*x^(18) + x^(17) + 3*x^(16) + 5*x^(15) + 2*x^(14) + 6*x^(13) + 3*x^(12) + 5*x^(11) + 2*x^(10) + 4*x^9 + 4*x^7 + 5*x^6 + 3*x^4 + 3*x^3 + 5*x^2 + 5*x + 6


### Algorytm Cantora i Zassenhausa dla ciała $\mathbb{F}_q$, gdzie $q=2^n$

Zobacz: Joachim von zur Gathen, Jurgen Gerhard, "Modern computer algebra", zadanie 14.16 (iii), str. 423.